## 05. 학습하기

In [10]:
import librosa
import librosa.display
import numpy as np

#import matplotlib.pyplot as plt

In [11]:
# 입력변수와 출력변수 생성하기

X_train=np.zeros((40,20))                # 입력변수 생성
y_train=np.zeros(40)                     # 출력변수 생성

#인덱스번호 0~19번까지는 레이블 1(배고픔), 21~40번까지는 레이블 0(웃음)
y_train[0:20] = 1

In [12]:
# hungry_특징추출  
for i in range(20):
  audio_path = '/Users/sunghyoukchoi/Google Drive/Code_of_textbook/수학과_함께하는_AI기초/3단원_0822/3-4/hungry/hungry_'+str(i+1)+'.wav' # 저장되어 있는 경로 확인
  y, sr = librosa.load(audio_path)
  mfcc= librosa.feature.mfcc(y=y, sr=sr)
  temp=mfcc.mean(axis=1)
  X_train[i]=temp                       # 인덱스번호 0~19번까지 

# laugh_특징추출  
for i in range(20):
  audio_path = '/Users/sunghyoukchoi/Google Drive/Code_of_textbook/수학과_함께하는_AI기초/3단원_0822/3-4/laugh/laugh_'+str(i+1)+'.wav' # 저장되어 있는 경로 확인
  y, sr = librosa.load(audio_path)
  mfcc= librosa.feature.mfcc(y=y, sr=sr)
  temp=mfcc.mean(axis=1) 
  X_train[i+20]=temp                   # 인덱스번호 20~39번까지

In [13]:
# 추출한 특성 값들을 데이터 셋으로 묶기

# 데이터 셋
data_sets=np.zeros((40,21))
data_sets[:,0:20]=X_train
data_sets[:,20]=y_train

In [18]:
# 특성의 여러 값 중 하나로 관측치 별 특성 간의 변화율을 대표값으로 선택 -- 의외로 잘 이해 안 감
def AccumAscentCurv(A):
    accVal = 0
    for i in range(len(A)-1):
        accVal = accVal + abs(A[i+1] - A[i])
    return accVal

X = [AccumAscentCurv(X_train[i]) for i in range(40)]
y = y_train

### 5-6 프로그램으로 해결하기
#### 5-6-1. 모델 학습하기 

In [19]:
# 데이터 준비하기
X=np.array(X).reshape(40,1)
y=y.reshape(40,1)

print(X[:5])
print(y[:5])

[[1065.84900665]
 [ 835.66071713]
 [ 874.80856338]
 [ 955.22123384]
 [ 965.16731071]]
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [20]:
def sigmoid(X):
    return 1/(1+np.exp(-X))

def cost_func(X, a):
    delta = 1e-7
    temp = beta0 + np.dot(X, beta1)
    Y_pred = sigmoid(temp)
    return -np.sum(a * np.log(Y_pred + delta) + (1-a) * np.log(1 - Y_pred) + delta)

In [24]:
# 에포크(Epoch) 마다 손실(비용)값을 계산하는 Error 함수 정의하기
def Error(X, a):
    delta = 1e-7    # log 무한대 발산 방지
    
    #temp = beta0 + X*beta1
    temp = beta0 + np.dot(X,beta1)
    Y_pred = sigmoid(temp)
    
    # likelihood 
    return  -np.sum( a*np.log(Y_pred + delta) + (1-a)*np.log((1 - Y_pred)+delta ) )

In [21]:
def predict(X):
    temp = np.dot(X, beta1) + beta0
    Y_pred = sigmoid(temp)
    if Y_pred >= 0.79:
        result = 1
    else:
        result = 0
    return Y_pred, result

In [22]:
# 편미분 함수 정의하기 -- 책에 별도의 설명이 없음
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

In [25]:
# 파라미터 업데이트하기 

learning_rate = 1e-3 
beta1 = np.random.rand(1,1)  
beta0 = np.random.rand(1)

F = lambda X : cost_func(X,y) 

for step in  range(10000001):  
    
    beta1 -= learning_rate * numerical_derivative(F, beta1)
    
    beta0 -= learning_rate * numerical_derivative(F, beta0)
    
    if (step % 100000 == 0):
        print("Epoch = ", step, "error value = ", Error(X,y) )

Epoch =  0 error value =  322.3619110191665
Epoch =  100000 error value =  196.646450648938
Epoch =  200000 error value =  139.20326177882035
Epoch =  300000 error value =  113.7460795139475
Epoch =  400000 error value =  98.61706183098735
Epoch =  500000 error value =  88.34350200878401
Epoch =  600000 error value =  80.80785952868752
Epoch =  700000 error value =  74.99436364348375
Epoch =  800000 error value =  70.34653350570878
Epoch =  900000 error value =  66.53044942262773
Epoch =  1000000 error value =  63.331955843218466
Epoch =  1100000 error value =  60.60654305829521
Epoch =  1200000 error value =  58.252702423050536
Epoch =  1300000 error value =  56.19677284219049
Epoch =  1400000 error value =  54.383843337020416
Epoch =  1500000 error value =  52.772043317818444
Epoch =  1600000 error value =  51.3288239209287
Epoch =  1700000 error value =  50.02845751141967
Epoch =  1800000 error value =  48.85031065527468
Epoch =  1900000 error value =  47.777622069741156
Epoch =  20

#### 5-6-2. 평가하기

In [26]:
count=0
for i in range(len(X)):
  (temp1, temp2) = predict(X[i])
  print(i+1, temp1,temp2,temp2 == y[i] )
  if (temp2==y[i]) :
    count += 1
print("정확도:{0}".format(count/len(X)))

1 [0.84336417] 1 [ True]
2 [0.78916508] 0 [False]
3 [0.79926916] 1 [ True]
4 [0.81887472] 1 [ True]
5 [0.82119308] 1 [ True]
6 [0.83976476] 1 [ True]
7 [0.83127172] 1 [ True]
8 [0.84021408] 1 [ True]
9 [0.83230874] 1 [ True]
10 [0.82025807] 1 [ True]
11 [0.84351907] 1 [ True]
12 [0.83001149] 1 [ True]
13 [0.81786718] 1 [ True]
14 [0.9072773] 1 [ True]
15 [0.78151714] 0 [False]
16 [0.86169475] 1 [ True]
17 [0.82637209] 1 [ True]
18 [0.84248312] 1 [ True]
19 [0.7887331] 0 [False]
20 [0.79188992] 1 [ True]
21 [0.71352151] 0 [ True]
22 [0.71439651] 0 [ True]
23 [0.71196897] 0 [ True]
24 [0.71266056] 0 [ True]
25 [0.71155582] 0 [ True]
26 [0.71040881] 0 [ True]
27 [0.71195771] 0 [ True]
28 [0.72010616] 0 [ True]
29 [0.75415447] 0 [ True]
30 [0.75415677] 0 [ True]
31 [0.75331426] 0 [ True]
32 [0.74846019] 0 [ True]
33 [0.74968151] 0 [ True]
34 [0.72548404] 0 [ True]
35 [0.72783728] 0 [ True]
36 [0.72597113] 0 [ True]
37 [0.72826685] 0 [ True]
38 [0.72810624] 0 [ True]
39 [0.7223566] 0 [ True

#### 5-6-3. 문제 해결하기

In [27]:
# test data 준비하기
audio_path = '/Users/sunghyoukchoi/Google Drive/Code_of_textbook/수학과_함께하는_AI기초/3단원_0822/3-4/test01.wav' # 폴더 확인 하기
y, sr = librosa.load(audio_path)
mfcc= librosa.feature.mfcc(y=y, sr=sr)
X_test=mfcc.mean(axis=1)

print(X_test)

[-297.9742      146.2829      -55.88721     -10.3654585    -0.58388543
  -46.56438      -8.005968      6.911135    -12.364081      2.9450808
   -2.4025762   -14.017911     -1.035388     -4.6966343    -2.4477491
    8.988057     -5.497288     -8.135314      1.624709     -9.138424  ]


In [28]:
X_test = np.array(AccumAscentCurv(X_test).reshape(1, 1))
print(X_test)

Y_pred, label = predict(X_test)
print(Y_pred, label)

[[920.70890856]]
[[0.81064874]] 1


#### scikit learn으로 로지스틱 모델 만들어 보기

In [31]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X, y_train)

Y_pred_prob = lr.predict_proba(X_test)
print(Y_pred_prob)

Y_pred_sk = lr.predict(X_test)
print(Y_pred_sk)

[[9.17064202e-11 1.00000000e+00]]
[1.]
